# Time Tracking

Our target user is a freelancer who bills clients by the time worked. Therefore, managing, tracking and planning your work time is a central task. Tuttle wants to make this as easy and effective as possible for you. Let's get started:

## Preamble

In [17]:
from pathlib import Path
import ipywidgets
from IPython import display

In [18]:
import tuttle

In [19]:
app = tuttle.app.App()

2022-02-19 16:16:49,645 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-02-19 16:16:49,646 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-02-19 16:16:49,647 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-19 16:16:49,649 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-02-19 16:16:49,650 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-19 16:16:49,651 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("icloudaccount")
2022-02-19 16:16:49,651 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-19 16:16:49,652 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("googleaccount")
2022-02-19 16:16:49,652 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-19 16:16:49,653 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bank")
2022-02-19 16:16:49,653 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-02-19 16:16:49,654 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2022-02-19 16:16:49,655 INFO sqlalchemy.engine.Engine [r

## How to Record Project Time with Tuttle

- Previously we asked you to assign a unique **tag** to each project that you want to track. We use this tag to assign time tracking entries to projects, trying to find the tag in:
    - the title of your calendar entries
    - the title of your time tracking entries

## Importing Time Tracking Data

Currently we provide several ways of inputting time management data:
    
1. **Cloud Calendar**: Querying your cloud calendar: Log into your cloud calendar provider and import your calendar events seamlessly.
2. **File Calendar**: Export your calendar in the iCalendar format and parse it.
3. **Time Tracking Tables**: You may prefer to track your time not in the calendar but a specialized tool. As long as this tool can export time tracking data in a consistent format, we want to provide an option to import it into Tuttle.

In [20]:
time_tracking_preference = ipywidgets.RadioButtons(options=["Cloud Calendar", "File Calendar", "Table"])

_Select your prereference for importing time tracking data:_

In [21]:
time_tracking_preference

RadioButtons(options=('Cloud Calendar', 'File Calendar', 'Table'), value='Cloud Calendar')

### A) Time Tracking via Cloud Calendar

If you have an Apple iCloud account, follow this process. (We are working on connectors with other cloud services)

In [23]:
icloud_username = app.user.icloud_account.user_name
icloud_username

2022-02-19 16:16:55,142 INFO sqlalchemy.engine.Engine SELECT user.id, user.name, user.subtitle, user.website, user.e_mail, user.phone_number, user.address_id, user."VAT_number", user.icloud_account_id, user.bank_account_id, user.logo 
FROM user
2022-02-19 16:16:55,144 INFO sqlalchemy.engine.Engine [no key 0.00169s] ()
2022-02-19 16:16:55,146 INFO sqlalchemy.engine.Engine SELECT icloudaccount.id AS icloudaccount_id, icloudaccount.user_name AS icloudaccount_user_name 
FROM icloudaccount 
WHERE icloudaccount.id = ?
2022-02-19 16:16:55,147 INFO sqlalchemy.engine.Engine [cached since 3.695s ago] (1,)


'mail@clstaudt.me'

Connect to your cloud calendar that contains your time tracking events:

In [24]:
from tuttle.calendar import ICloudCalendar

In [25]:
if time_tracking_preference.value == "Cloud Calendar":
    my_calendar = ICloudCalendar(
        icloud=tuttle.cloud.login_iCloud(
            user_name=icloud_username,
        ),
        name=input("calendar name: "),
    )

### B) Time  Tracking via Calendar Import

In [26]:
from tuttle.calendar import FileCalendar

**Example: file calendar**

In [27]:
timetracking_calendar_path = Path("../../tests/data/TuttleDemo-TimeTracking.ics")

In [28]:
if time_tracking_preference.value == "File Calendar":
    my_calendar = FileCalendar(
        path=timetracking_calendar_path, 
        name="TimeTracking"
    )

### C) Time Tracking via Data Export from Time Tracking Tools

![](https://media.gettyimages.com/vectors/under-construction-sign-in-white-background-vector-id536013219?b=1&k=6&m=536013219&s=170x170&h=_-DZGgLDo-84eyZSJZmgwlekma7FYt3LdOdLIC7qktw=)

## Generating Time Sheets

Now that we have connected a source for time tracking data, we can pass it to other functions. In the following we generate **time sheets**. 

A **time sheet** 
- can be rendered to a document for reporting purposes
- is the basis for an **invoice** - more on invoicing later.

**Select a project**

**Example**

In [29]:
my_project = app.get_project(title="Heating Repair")

2022-02-19 16:16:59,244 INFO sqlalchemy.engine.Engine SELECT project.id, project.title, project.tag, project.start_date, project.end_date, project.contract_id 
FROM project 
WHERE project.title = ?
2022-02-19 16:16:59,246 INFO sqlalchemy.engine.Engine [no key 0.00270s] ('Heating Repair',)


_Your turn: Select one of your projects by title_

**Generate a time sheet**

In [30]:
my_timesheet = tuttle.timetracking.generate_timesheet(
    source=my_calendar,
    project=my_project,
    period="February 2022",
    item_description=my_project.title,
)

2022-02-19 16:16:59,966 INFO sqlalchemy.engine.Engine SELECT contract.unit AS contract_unit, contract.billing_cycle AS contract_billing_cycle, contract.id AS contract_id, contract.title AS contract_title, contract.signature_date AS contract_signature_date, contract.start_date AS contract_start_date, contract.end_date AS contract_end_date, contract.client_id AS contract_client_id, contract.rate AS contract_rate, contract.currency AS contract_currency, contract.volume AS contract_volume, contract.units_per_workday AS contract_units_per_workday, contract.term_of_payment AS contract_term_of_payment 
FROM contract 
WHERE contract.id = ?
2022-02-19 16:16:59,967 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (1,)


/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:101: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return super().execute(  # type: ignore


_Your turn: Generate a timesheet for your project_

**Display the timesheet data**

In [31]:
my_timesheet.table

,id,timesheet_id,begin,end,duration,title,tag,description
0,None,None,2022-02-17 01:00:00+01:00,2022-02-18 01:00:00+01:00,0 days 08:00:00,#HeatingRepair,#HeatingRepair,Heating Repair
1,None,None,2022-02-18 01:00:00+01:00,2022-02-19 01:00:00+01:00,0 days 08:00:00,#HeatingRepair,#HeatingRepair,Heating Repair


**Render the timesheet document**

The following function renders a timesheet to an HTML-based layout:

In [32]:
display.HTML(
    tuttle.rendering.render_timesheet(
        user=app.user,
        timesheet=my_timesheet,
        style=None,
    )
)

2022-02-19 16:17:01,655 INFO sqlalchemy.engine.Engine INSERT INTO timesheet (title, date, period, project_id, comment) VALUES (?, ?, ?, ?, ?)
2022-02-19 16:17:01,657 INFO sqlalchemy.engine.Engine [generated in 0.00178s] ('Heating Repair - February 2022', '2022-02-19', 'February 2022', 1, '')
2022-02-19 16:17:01,660 INFO sqlalchemy.engine.Engine INSERT INTO timetrackingitem (timesheet_id, "begin", "end", duration, title, tag, description) VALUES (?, ?, ?, ?, ?, ?, ?)
2022-02-19 16:17:01,661 INFO sqlalchemy.engine.Engine [generated in 0.00077s] (1, '2022-02-17 01:00:00.000000', '2022-02-18 01:00:00.000000', '1970-01-01 08:00:00.000000', '#HeatingRepair', '#HeatingRepair', 'Heating Repair')
2022-02-19 16:17:01,662 INFO sqlalchemy.engine.Engine INSERT INTO timetrackingitem (timesheet_id, "begin", "end", duration, title, tag, description) VALUES (?, ?, ?, ?, ?, ?, ?)
2022-02-19 16:17:01,663 INFO sqlalchemy.engine.Engine [cached since 0.002185s ago] (1, '2022-02-18 01:00:00.000000', '2022-02

Rendering to a HTML file with various styles:

![](https://media.gettyimages.com/vectors/under-construction-sign-in-white-background-vector-id536013219?b=1&k=6&m=536013219&s=170x170&h=_-DZGgLDo-84eyZSJZmgwlekma7FYt3LdOdLIC7qktw=)